In [1]:
import skfuzzy
import numpy as np
from utils.datastore import *

In [2]:
ds = DataStore()
D = ds.build_datasets(labeled=False)

In [ ]:
# Hyperparameters:
metric = 'cityblock'
ncenters = 7
error = 1e-06
niter = 50
fuzzy_param = [1.1,1.6,2.0]

best_results = []

for Di in D:
    results = []
    J = np.nan
    best_result_index = 0
    for i in range(niter):
        results.append(
            skfuzzy.cluster.cmeans(Di.transpose(), ncenters, 2, error=error, metric=metric, 
                    maxiter=1000, init=None)
        )
        jm = results[i][4][-1]
        if np.isnan(J):
            J = jm
        else:
            if jm < J:
                best_result_index = i
    best_results.append(results[best_result_index])

        

In [9]:
# Modified Partition Coefficient
def modified_partition_coefficient(ncenters, partition_coefficient):
    return 1 - (ncenters/(ncenters+1))*(1-partition_coefficient)

# Partition Entropy
def partition_entropy(U):
    C, N = U.shape
    s = 0
    for i in range(C):
        for j in range(N):
            s += U[i][j] * np.log(U[i][j])
    return -(1/N)*s
    
for i in range(3):
    print("Partition Entropy D{0}: {1}".format(i,partition_entropy(best_results[i][1])))
    print("Modified Partition Coefficient D{0}: {1}".format(i,modified_partition_coefficient(7,best_results[i][-1])))
    

    

Partition Entropy D0: 0.23302030014392214
Modified Partition Coefficient D0: 0.8859173509763976
Partition Entropy D1: 0.599264319663365
Modified Partition Coefficient D1: 0.75332186560258
Partition Entropy D2: 0.7271844668329119
Modified Partition Coefficient D2: 0.7008406561155085
